In [124]:
import ctypes
import numpy as np
import math
import time

#Define point structure
class HeliosPoint(ctypes.Structure):
    #_pack_=1
    _fields_ = [('x', ctypes.c_uint16),
                ('y', ctypes.c_uint16),
                ('r', ctypes.c_uint8),
                ('g', ctypes.c_uint8),
                ('b', ctypes.c_uint8),
                ('i', ctypes.c_uint8)]
class Dac:
    def __init__(self):
        #Load and initialize library
        self.HeliosLib = ctypes.cdll.LoadLibrary("./libHeliosDacAPI.so")
        self.num_devices = self.HeliosLib.OpenDevices()
        print("Found ", self.num_devices, "Helios DACs")
        # Define limits
        self.xy_max = int(2**12-1)

class DacQueue:
    '''
    A queue for patterns sent to the dac. 
    Performs smart stitching between patterns.
    '''
    def __init__(self):
        # Dac object for this queue
        self.dac = Dac()
        # The last position of the last pattern (x,y)
        self.last_pos = (0,0)
        # Sample rate of the DAC
        self.dac_rate = 55000
        # Debugging dac rate
        self.debug_rate = 550
        
    
    def submit(self, pat_pos, pat_col, angular_density=100, debug=False):
        '''
        Submits a new pattern to the dac, transitioning smoothly from the last one.
        Angular density describes how many points per radian should be used to transition
        '''
        # Make the transition
        dist = np.sqrt(np.sum(np.power(self.last_pos-pat_pos[0,:], 2)))
        num_gap_points = int(dist*angular_density)

        gap_pos = np.linspace(self.last_pos, pat_pos[0,:], num_gap_points, dtype=np.float)
        gap_col = np.zeros((num_gap_points, 3), dtype=np.float)
        if(debug):
            gap_col = np.ones_like(gap_col)/4

        # Prep the transition pattern
        gap_points, num_gap_points = self.prep_pattern(gap_pos, gap_col, gap=True)
        
        # Prep the new pattern
        pat_points, num_pat_points = self.prep_pattern(pat_pos, pat_col)
        
        # Send the transition pattern to the dac
        self.write_frames(gap_points, num_gap_points, do_not_loop=True, debug=debug)
        # Send the new pattern to the dac
        self.write_frames(pat_points, num_pat_points, do_not_loop=True, debug=debug)
        # Set the last_position
        self.last_pos = pat_pos[-1, :].copy()
        
    def prep_pattern(self, arr_pos, arr_col, gap=False):
        '''
        1) Scales and color corrects 
        2) Converts from unit space to DAC coordinates
        3) Produces a DAC compatible frame and displays it
        '''
        # Force pass-by-value
        arr_pos = arr_pos.copy()
        arr_col = arr_col.copy()
        # Scale the pattern 
        arr_pos = position_corection(arr_pos)
        # Format the position array (-1.0-1.0) -> int(0-4095)
        arr_pos = (((arr_pos+1)/2.0)*self.dac.xy_max).astype(np.int32)

        # Perform color correction if not gap points
        if(not gap):
            arr_col = color_correction(arr_col)
        # Format the color array (0-1.0) -> int(0-255)
        arr_col = (arr_col*255).astype(np.int32)
        # Fill a heliospoint arr with these values
        num_points = len(arr_pos)

        frameType = HeliosPoint * num_points
        points = frameType()
        # Fill the frame
        for idx in range(num_points):
            points[idx] =     HeliosPoint(int(arr_pos[idx, 0]), 
                                         int(arr_pos[idx, 1]), 
                                         int(arr_col[idx, 0]), 
                                         int(arr_col[idx, 1]),
                                         int(arr_col[idx, 2]),
                                         int(255))
        
        return points, num_points
            
    def write_frames(self, points, num_points, do_not_loop=False, start_immediately=False, debug=False):
        if(num_points < 1):
            return
        # Write the values out 
        status_attempts = 0
        max_attempts = np.inf
        # Make 512 attempts for DAC status to be ready. After that, just give up and try to write the frame anyway
        while(status_attempts < max_attempts and self.dac.HeliosLib.GetStatus(0) != 1):
            status_attempts += 1
            
        # Create flags 
        flags = do_not_loop << 1 | start_immediately << 0
        # Send to DAC
        frame_rate = self.debug_rate if debug else self.dac_rate
        self.dac.HeliosLib.WriteFrame(0, frame_rate, flags, ctypes.pointer(points), num_points)



In [125]:
def color_correction(arr_color):
    '''
    Corrects the nonlinearities in the color curve 
    '''
    #arr_color = np.exp(np.log(2)*np.power(arr_color,1)) - 1
    # Scale to the lower cutoff (R, G, B)
    lower_cutoff = np.array([0.25, 0.09, 0.09])
    arr_color = arr_color*(1-lower_cutoff) + lower_cutoff
    
    return arr_color

def position_corection(arr_pos):
    '''
    Corrects x-y directions so that (x=-1,y=-1) is bottom left
    Scales the entire pattern so that it does not clip (amps are weird)
    '''
    # Reverse direction of x and y
    arr_pos *= -1
    
    max_scale = 0.75
    return arr_pos*max_scale
    
def make_circular(arr_pos, arr_color):
    return np.concatenate([arr_pos, arr_pos[::-1, :]]), np.concatenate([arr_color, arr_color[::-1, :]])


In [126]:
queue = DacQueue()

Found  1 Helios DACs


In [60]:
# Produce colored circles 
T = 1000
theta = np.linspace(0, 2*np.pi, T)
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.cos(theta)
pos_arr[:, 1] = np.sin(theta)

start = (0.1, 1.0, 0.1)
finish = (1.0, 0.1, 1.0)
smooth = np.linspace(start, finish, T//2)

color_arr = np.concatenate([smooth, smooth[::-1]])/4

for i in range(100):
    queue.submit(pos_arr, color_arr)

    queue.submit(pos_arr[:, ::-1], color_arr)

    queue.submit(pos_arr, color_arr)

In [61]:
# Produce a SINE wave
T = 1000
theta = np.linspace(0, 2*np.pi, T)
freq = 3
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.linspace(-1.0, 1.0, T)
pos_arr[:, 1] = np.sin(theta*freq)

start = (0.0, 0.0, 1.0)
finish = (1.0, 0.0, 1.0)
smooth = np.linspace(start, finish, T)

color_arr = np.concatenate([smooth, smooth[::-1]])/4

for i in range(100):
    queue.submit(pos_arr, color_arr, debug=False)


In [133]:
# Produce a phase shifting sine wave
T = 600
freq = 4
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.linspace(-1.0, 1.0, T)
start = (0.0, 0.0, 1.0)
finish = (1.0, 1.0, 1.0)
smooth = np.linspace(start, finish, T)

color_arr = np.concatenate([smooth, smooth[::-1]])/4

while(True):
    for phi in np.linspace(0, 2*np.pi, 100):
        theta = np.linspace(phi, phi+2*np.pi, T)

        pos_arr[:, 1] = np.sin(theta*freq)

        queue.submit(pos_arr, color_arr, debug=False)


KeyboardInterrupt: 

In [103]:
# Produce a double SINE wave
T = 1000
theta = np.linspace(0, 4*np.pi, T)
freq = 3
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.concatenate([np.linspace(-1.0, 1.0, T//2), np.linspace(1.0, -1.0, T//2)])
pos_arr[:, 1] = np.sin(theta*freq)

start = (0.0, 0.0, 1.0)
finish = (1.0, 0.0, 1.0)
smooth = np.linspace(start, finish, T)

color_arr = np.concatenate([smooth, smooth[::-1]])/4

for i in range(1000):
    queue.submit(pos_arr, color_arr, debug=False)


[-1.         -0.94500082] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616

[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-

[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-15] [-1.  0.]
[-1.00000000e+00 -1.46957616e-15]
[-1.00000000e+00 -1.46957616e-

KeyboardInterrupt: 

In [111]:
# Produce a modulated wave
T = 1000
theta = np.linspace(0, 4*np.pi, T)
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.concatenate([np.linspace(-1.0, 1.0, T//2), np.linspace(1.0, -1.0, T//2)])
start = (0.0, 0.0, 1.0)
finish = (1.0, 0.0, 1.0)
smooth = np.linspace(start, finish, T)

color_arr = np.concatenate([smooth, smooth[::-1]])/4
for i in range(100):
    for freq in np.linspace(1, 5, 100):
        pos_arr[:, 1] = np.sin(theta*freq)
        queue.submit(pos_arr, color_arr, debug=False)


[-1.          0.77614646] [-1.  0.]
[-1.0000000e+00 -4.8985872e-16]
[-1.          0.48619674] [-1.  0.]
[-1.          0.48619674]
[-1.          0.84972543] [-1.  0.]
[-1.          0.84972543]
[-1.          0.99886734] [-1.  0.]
[-1.          0.99886734]
[-1.          0.89599377] [-1.  0.]
[-1.          0.89599377]
[-1.          0.56705986] [-1.  0.]
[-1.          0.56705986]
[-1.          0.09505604] [-1.  0.]
[-1.          0.09505604]
[-1.         -0.40093054] [-1.  0.]
[-1.         -0.40093054]
[-1.         -0.79576184] [-1.  0.]
[-1.         -0.79576184]
[-1.         -0.98982144] [-1.  0.]
[-1.         -0.98982144]
[-1.         -0.93414786] [-1.  0.]
[-1.         -0.93414786]
[-1.         -0.64278761] [-1.  0.]
[-1.         -0.64278761]
[-1.         -0.18925124] [-1.  0.]
[-1.         -0.18925124]
[-1.          0.31203345] [-1.  0.]
[-1.          0.31203345]
[-1.          0.73459171] [-1.  0.]
[-1.          0.73459171]
[-1.          0.97181157] [-1.  0.]
[-1.          0.97181157]
[-

[-1.         -0.54064082]
[-1.         -0.06342392] [-1.  0.]
[-1.         -0.06342392]
[-1.          0.42979491] [-1.  0.]
[-1.          0.42979491]
[-1.          0.81457595] [-1.  0.]
[-1.          0.81457595]
[-1.          0.99383846] [-1.  0.]
[-1.          0.99383846]
[-1.          0.92235429] [-1.  0.]
[-1.          0.92235429]
[-1.          0.61815899] [-1.  0.]
[-1.          0.61815899]
[-1.         0.1580014] [-1.  0.]
[-1.         0.1580014]
[-1.         -0.34202014] [-1.  0.]
[-1.         -0.34202014]
[-1.         -0.75574957] [-1.  0.]
[-1.         -0.75574957]
[-1.         -0.97880245] [-1.  0.]
[-1.         -0.97880245]
[-1.         -0.95490224] [-1.  0.]
[-1.         -0.95490224]
[-1.         -0.69007901] [-1.  0.]
[-1.         -0.69007901]
[-1.         -0.25114799] [-1.  0.]
[-1.         -0.25114799]
[-1.          0.25114799] [-1.  0.]
[-1.          0.25114799]
[-1.          0.69007901] [-1.  0.]
[-1.          0.69007901]
[-1.          0.95490224] [-1.  0.]
[-1.        

KeyboardInterrupt: 

In [11]:
# Make a tri-colored circle
# Make a tricolor circle
T = 1000
pos_arr = np.zeros((T, 2))
col_arr = np.zeros((T, 3))

# Set Xs and Ys
theta = np.linspace(0, 2*np.pi, T)
pos_arr[:,0] = np.cos(theta)
pos_arr[:,1] = np.sin(theta)


# Tri color the circle
col_arr[theta < np.pi, 0] = 0
col_arr[theta < np.pi, 1] = 0
col_arr[theta < np.pi, 2] = 0.3
col_arr[theta < (2/3)*np.pi, 0] = 0
col_arr[theta < (2/3)*np.pi, 1] = 0.3
col_arr[theta < (2/3)*np.pi, 2] = 0
col_arr[theta < (1/3)*np.pi, 0] = 0.3
col_arr[theta < (1/3)*np.pi, 1] = 0
col_arr[theta < (1/3)*np.pi, 2] = 0

for i in range(100):
    queue.submit(pos_arr, color_arr)


In [101]:
scales = np.concatenate([np.linspace(1.0, 0.5, 10), np.linspace(0.5, 1, 20)])

import time
while(True):
    for scale in scales:
        new_pos_arr = scale*pos_arr
        queue.submit(new_pos_arr, col_arr)
    break


(0, 0) [-1.         -0.34202014]
[ 1.         -0.34202014]
[ 1.         -0.34202014] [-0.94444444 -0.32301902]
[ 0.94444444 -0.32301902]
[ 0.94444444 -0.32301902] [-0.88888889 -0.30401791]
[ 0.88888889 -0.30401791]
[ 0.88888889 -0.30401791] [-0.83333333 -0.28501679]
[ 0.83333333 -0.28501679]
[ 0.83333333 -0.28501679] [-0.77777778 -0.26601567]
[ 0.77777778 -0.26601567]
[ 0.77777778 -0.26601567] [-0.72222222 -0.24701455]
[ 0.72222222 -0.24701455]
[ 0.72222222 -0.24701455] [-0.66666667 -0.22801343]
[ 0.66666667 -0.22801343]
[ 0.66666667 -0.22801343] [-0.61111111 -0.20901231]
[ 0.61111111 -0.20901231]
[ 0.61111111 -0.20901231] [-0.55555556 -0.19001119]
[ 0.55555556 -0.19001119]
[ 0.55555556 -0.19001119] [-0.5        -0.17101007]
[ 0.5        -0.17101007]
[ 0.5        -0.17101007] [-0.5        -0.17101007]
[ 0.5        -0.17101007]
[ 0.5        -0.17101007] [-0.52631579 -0.1800106 ]
[ 0.52631579 -0.1800106 ]
[ 0.52631579 -0.1800106 ] [-0.55263158 -0.18901113]
[ 0.55263158 -0.18901113]
[ 0.5

a:  (1000, 2) (1000, 3)


NameError: name 'stitch_patterns' is not defined

In [ ]:
import cv2
def draw_bitmap(arr_pos, arr_color, dims=(400,400)):
    '''
    Draws a pattern as if we were drawing it on a wall
    '''
    # Copy the arrays so we don't change them 
    arr_pos = arr_pos.copy()
    arr_color = arr_color.copy()
    # Map the positions
    arr_pos[:,0] *= -1
    arr_pos = (((arr_pos+1)/2)*dims).astype(np.int)
    # Map the colors
    arr_color = arr_color[:,::-1]
    T = len(arr_pos)
    img = np.zeros(dims+(3,))
    # Draw points
    for t in range(1, T):
        cv2.line(img, arr_pos[t-1], arr_pos[t], color=arr_color[t-1], thickness=2)
    cv2.imshow('img', img)
    cv2.waitKey(0)
    
    

In [97]:
# Make three parallel lines
T_line = 100

arr_pos_list = []
arr_color_list = []
for c_idx, y in enumerate(range(-1, 2, 1)):
    arr_pos_list.append(np.stack([np.linspace(-1, 1, T_line), y*np.ones(T_line)], axis=1))
    arr_color = np.zeros((T_line, 3))
    arr_color[:, c_idx] = np.linspace(0, 1, T_line)
    arr_color_list.append(arr_color.copy())

for i in range(100):
    for arr_pos, arr_color in zip(arr_pos_list, arr_color_list):
        queue.submit(arr_pos, arr_color)

[1. 1.] [-1. -1.]
[ 1. -1.]
[ 1. -1.] [-1.  0.]
[1. 0.]
[1. 0.] [-1.  1.]
[1. 1.]
[1. 1.] [-1. -1.]
[ 1. -1.]
[ 1. -1.] [-1.  0.]
[1. 0.]
[1. 0.] [-1.  1.]
[1. 1.]
[1. 1.] [-1. -1.]
[ 1. -1.]
[ 1. -1.] [-1.  0.]
[1. 0.]
[1. 0.] [-1.  1.]
[1. 1.]
[1. 1.] [-1. -1.]
[ 1. -1.]
[ 1. -1.] [-1.  0.]
[1. 0.]
[1. 0.] [-1.  1.]
[1. 1.]
[1. 1.] [-1. -1.]
[ 1. -1.]
[ 1. -1.] [-1.  0.]
[1. 0.]
[1. 0.] [-1.  1.]
[1. 1.]
[1. 1.] [-1. -1.]
[ 1. -1.]
[ 1. -1.] [-1.  0.]
[1. 0.]
[1. 0.] [-1.  1.]
[1. 1.]
[1. 1.] [-1. -1.]
[ 1. -1.]
[ 1. -1.] [-1.  0.]
[1. 0.]
[1. 0.] [-1.  1.]
[1. 1.]
[1. 1.] [-1. -1.]
[ 1. -1.]
[ 1. -1.] [-1.  0.]
[1. 0.]
[1. 0.] [-1.  1.]
[1. 1.]
[1. 1.] [-1. -1.]
[ 1. -1.]
[ 1. -1.] [-1.  0.]
[1. 0.]
[1. 0.] [-1.  1.]
[1. 1.]
[1. 1.] [-1. -1.]
[ 1. -1.]
[ 1. -1.] [-1.  0.]
[1. 0.]
[1. 0.] [-1.  1.]
[1. 1.]
[1. 1.] [-1. -1.]
[ 1. -1.]
[ 1. -1.] [-1.  0.]
[1. 0.]
[1. 0.] [-1.  1.]
[1. 1.]
[1. 1.] [-1. -1.]
[ 1. -1.]
[ 1. -1.] [-1.  0.]
[1. 0.]
[1. 0.] [-1.  1.]
[1. 1.]
[1. 1.] [-1. -1.

In [83]:
# Make three concentric circles
T_line = 200
theta_line = np.linspace(0, 2*np.pi, T_line)

arr_pos_list = []
arr_color_list = []
for c_idx, scale in enumerate([0.9, 0.95, 1]):
    arr_pos_list.append(np.stack([np.cos(theta_line), np.sin(theta_line)], axis=1)/scale)
    arr_color = np.zeros((T_line, 3))
    arr_color[:, c_idx] = np.linspace(0, 1, T_line)
    arr_color_list.append(arr_color.copy())

for i in range(1000):
    for arr_pos, arr_color in zip(arr_pos_list, arr_color_list):
        queue.submit(arr_pos, arr_color)

[ 1.         -0.34202014] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+0

[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -

[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] 

[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] 

[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -

...

[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] 

[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] 

[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] 

[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -

[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] [1.05263158 0.        ]
[ 1.05263158e+00 -2.57820379e-16]
[ 1.05263158e+00 -2.57820379e-16] [1. 0.]
[ 1.0000000e+00 -2.4492936e-16]
[ 1.0000000e+00 -2.4492936e-16] [1.11111111 0.        ]
[ 1.11111111e+00 -2.72143733e-16]
[ 1.11111111e+00 -2.72143733e-16] 

In [ ]:
draw_bitmap(arr_pos, arr_color)

In [92]:
# make a sine wave
T = 500
freq = 4
theta_line = np.linspace(0, 2*np.pi, T)
arr_pos = np.stack([np.linspace(-1, 1, T), np.sin(freq*theta_line)], axis=1)
#arr_color = np.ones((T, 3))
arr_color = np.linspace((0.1, 1.0, 0.3), (1.0, 0.0, 0.7), T)
for i in range(100):
    queue.submit(arr_pos, arr_color, debug=False)

[ 1.00000000e+00 -9.79717439e-16] [-1.  0.]
[ 1.00000000e+00 -9.79717439e-16]
[ 1.00000000e+00 -9.79717439e-16] [-1.  0.]
[ 1.00000000e+00 -9.79717439e-16]
[ 1.00000000e+00 -9.79717439e-16] [-1.  0.]
[ 1.00000000e+00 -9.79717439e-16]
[ 1.00000000e+00 -9.79717439e-16] [-1.  0.]
[ 1.00000000e+00 -9.79717439e-16]
[ 1.00000000e+00 -9.79717439e-16] [-1.  0.]
[ 1.00000000e+00 -9.79717439e-16]
[ 1.00000000e+00 -9.79717439e-16] [-1.  0.]
[ 1.00000000e+00 -9.79717439e-16]
[ 1.00000000e+00 -9.79717439e-16] [-1.  0.]
[ 1.00000000e+00 -9.79717439e-16]
[ 1.00000000e+00 -9.79717439e-16] [-1.  0.]
[ 1.00000000e+00 -9.79717439e-16]
[ 1.00000000e+00 -9.79717439e-16] [-1.  0.]
[ 1.00000000e+00 -9.79717439e-16]
[ 1.00000000e+00 -9.79717439e-16] [-1.  0.]
[ 1.00000000e+00 -9.79717439e-16]
[ 1.00000000e+00 -9.79717439e-16] [-1.  0.]
[ 1.00000000e+00 -9.79717439e-16]
[ 1.00000000e+00 -9.79717439e-16] [-1.  0.]
[ 1.00000000e+00 -9.79717439e-16]
[ 1.00000000e+00 -9.79717439e-16] [-1.  0.]
[ 1.00000000e+00 -9.

In [6]:
from skimage.color import rgb2lab, lab2rgb

In [17]:
print(lab2rgb((0,0,0)))
print(rgb2lab((0.10,0.10,0.255)))
print(rgb2lab((0.5,1,1)))


[0. 0. 0.]
[ 11.357263    13.5097441  -25.23433865]
[ 93.13904229 -35.33482179 -10.88977706]


In [415]:
# Circles in the CIELAB Space
# UNFINISHED - see this for documentation: https://github.com/scikit-image/scikit-image/blob/main/skimage/color/colorconv.py#L1142-L1183
# Range is here? :https://stackoverflow.com/questions/25294141/cielab-color-range-for-scikit-image
scales = np.concatenate([np.linspace(1, 0.1, 200), np.linspace(0.1, 1, 200)])

T = 400


# Set Xs and Ys
theta = np.linspace(0, 2*np.pi, T)
arr_pos = np.zeros((T, 2))
arr_pos[:,0] = np.cos(theta)
arr_pos[:,1] = np.sin(theta)

# Set the colors 
arr_cols = []
for scale in scales:
    arr_col_lab = np.zeros((T, 3))
    # Set the luminosity to the scale
    #arr_col_lab[:, 0] = scale*100.0
    arr_col_lab[:, 0] = 10
    arr_col_lab[:, 1::] = 127.0*arr_pos/2.0
    arr_cols.append(lab2rgb(arr_col_lab))

for i in range(1000):
    for scale, arr_col in zip(scales, arr_cols):
        queue.submit(arr_pos*scale, arr_col)

    
    

KeyboardInterrupt: 